In [31]:
import os
import time
import logging
import shutil
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup

def text_from_html(body):
    print("text_from_html called")
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll("div",text=re.compile('No data to display'))
    try:
        return print(u" ".join(t.strip() for t in texts))
    except Exception as e:
        return 0


def veh_id_extract (veh_id):
    """
    Input veh_id one vehicle at a time
    Date Range (start_date,end_date)
    """
    #  Pass vehicle id to ID input
    try:
        veh_id_input_element = driver.find_element(By.XPATH,"//input[@id='comboBoxVehicleSearch_I']")
        veh_id_input_element.clear()
        veh_id_input_element.send_keys(veh_id)
        
    except Exception as e:
        logging.info(f"{veh_id} could not be found in drop down/n {e}")
    
    #  Navigate to APPLY
    # wait for drop down list to load
    wait = WebDriverWait(driver, 30)
    time.sleep(10)

    driver.find_element(By.XPATH,"//btn[@onclick='historyFilter_Submit()']").click()

    # if text_from_html(driver.page_source) != 0:
    #     return

    # Wait for tables to load and export button to load
    time.sleep(30)

    # Navigate to EXPORT button
    try:
        wait.until(EC.presence_of_all_elements_located((By.XPATH,"//button[@type='button' and @aria-haspopup='true']")))

        export_button = driver.find_element(By.XPATH,"//button[contains(text(),'Export')]").click()
        export_csv = driver.find_element(By.XPATH,'//*[@id="historyParentCallback"]/div/div/div[4]/div/div/button[3]').click()
    except Exception as e:
        logging.info(f"Export button not clickable/n {e}")
        return print("Error - Ending routine")


    # Download buffer time
    time.sleep(20)
    logging.info("Downloading CSV file")

    # Rename CSV file and move to OUTPUT folder
    origin = r'C:\Users\MelissaLu\Downloads'
    target = r'C:\Users\MelissaLu\OneDrive - Metrolinx\Desktop\Projects\Mobile Eco-2 Scraping\output'

    downloaded_fn = "historyMasterGrid.csv"

    my_file = os.path.join(origin,downloaded_fn)

    target_loc = os.path.join(target,downloaded_fn)
    


    shutil.move(my_file, target_loc)

    os.chdir(target)

    os.rename("historyMasterGrid.csv" , f"historyMasterGrid_{veh_id}.csv")
    logging.info(f"Downloading historyMasterGrid_{veh_id}.csv is complete")

    time.sleep(30)

    return print(f'File downloaded for {veh_id}')


# initialize chrome driver
# TODO  DeprecationWarning: executable_path has been deprecated, please pass in a Service object
driver = webdriver.Chrome("chrome")
driver.implicitly_wait(20)

url = r"http://cavl-pvme201:8090/MOBILEvhm/Login?ReturnUrl=%2fMOBILEvhm%2fDevicesGroups%2fVHM" 
driver.get(url)

# Mobile-Eco-2 Credentials
username = "inita"
password = "init"

# Login
driver.find_element("id", "UserShortName").send_keys(username)
driver.find_element("id", "Password").send_keys(password)

driver.find_element(By.XPATH, "//div[input/@name='loginRequest']/button[1]").click()

# Navigate NAV for Dashboard | Control Board | Veh & Components | Realtime Cockpit | Config | Fleet History
URLS = {
        "fleetHistory":r"http://cavl-pvme201:8090/MOBILEvhm/DevicesGroups/FleetHistory",
        "vehHistory":r"http://cavl-pvme201:8090/MOBILEvhm/DevicesGroups/History",
        }

# Vehicle History
driver.get(URLS["vehHistory"])



# Get all vehicle ID
pg_src = driver.page_source
soup = BeautifulSoup(pg_src, 'html.parser')
veh_id_bs_list= soup.find_all(class_="dxeListBoxItem_DevEx dxeLTM")

# TODO check veh id list
veh_id_list = []
for id in veh_id_bs_list:
    veh_id_list.append(id.get_text())

# Extract CSV files
start_i = 140
end_i = 545+1
veh_id_sub = veh_id_list[start_i:end_i]
for id in veh_id_sub:
    print(id)
    if len(id) < 2:
        continue
    else:
        veh_id_extract (id)
        


driver.quit()





C:\Users\MelissaLu\AppData\Local\Temp\1\ipykernel_26844\4092354426.py:94: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chrome")


8420


KeyboardInterrupt: 

In [29]:

for t in veh_id_bs_list:
    if t.text == "8420":
        print(veh_id_bs_list.index(t))

print(len(veh_id_bs_list))


140
545
